In [1]:
import numpy as np
import pandas as pd
import pyodbc
import warnings

warnings.filterwarnings("ignore")

In [2]:
def get_game_category():
    cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
    query = f"""
    SET NOCOUNT ON;
    DROP TABLE IF EXISTS #cg
    SELECT cg.GameID,
        cg.GameCategoryID,
        cg.GameProviderID,
        cg.Name_en
    INTO #cg
    FROM C_Game cg

    DROP TABLE IF EXISTS #gc
    SELECT gc.GameCategoryID,
        gc.GameCategoryName
    INTO #gc
    FROM C_GameCategory gc

    DROP TABLE IF EXISTS #gp
    SELECT gp.GameProviderID,
        gp.GameProviderName
    INTO #gp
    FROM C_GameProvider gp

    DROP TABLE IF EXISTS #a
    SELECT #cg.GameID,
        #cg.Name_en,
        #gc.GameCategoryName,
        #gp.GameProviderName
    INTO #a
    FROM #cg
    LEFT JOIN #gc ON #cg.GameCategoryID = #gc.GameCategoryID
    LEFT JOIN #gp ON #cg.GameProviderID = #gp.GameProviderID

    DROP TABLE IF EXISTS #cas
    SELECT u.PartnerUserID,
        count(*) as BetCount,
        a.BetAmount,
        g.GameID
    INTO #cas
    FROM (
    SELECT o.UserID,
        o.GameID,
        (IIF(o.CalculationDate < '2021-03-01', o.OrderAmount,
        CASE WHEN cg.GameProviderID IN (48, 10) AND o.TypeId IN (1, 5, 8, 18, 33) THEN o.OrderAmount
        WHEN cg.GameProviderID NOT IN (48, 10) THEN o.OrderAmount ELSE 0 END)) AS BetAmount
    FROM casino.orders o
    INNER JOIN C_Game cg ON cg.GameID = o.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON o.UserID = u.UserID
    INNER JOIN C_GameProvider gp ON gp.GameProviderID = cg.GameProviderID
    WHERE o.CalculationDate_DT >= '2022-12-01'
    AND gp.GameProviderID <> 3
    AND OrderStateID IN (2, 3, 5, 6)
    AND o.OperationTypeID IN (3, 299)
    AND IIF(cg.GameProviderID IN (48, 10)
    AND o.CalculationDate < '2021-03-01', o.TypeId, 0) IN (0, 1, 5, 8, 18, 33)
    ) a
    INNER JOIN C_Game g ON g.GameID = a.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON u.UserID = a.UserID
    WHERE u.isDeleted = 0
    AND u.UserTypeID NOT IN (1, 20, 3, 21)
    AND u.UserName NOT LIKE 'test%'
    AND u.UserName NOT LIKE '%TestClient%'
    GROUP BY u.PartnerUserID,
        a.BetAmount,
        g.GameID

    SELECT * FROM (SELECT #cas.PartnerUserId AS TOTOID,
        SUM(#cas.BetCount) AS Bet_count,
        SUM(#cas.BetAmount) AS Bet_amount,
        #b.Type AS Category
    FROM #cas
    LEFT JOIN (SELECT *, (
    CASE
				WHEN #a.Name_en LIKE '%HabaneroJackpot%'
					THEN 'Other'
				WHEN #a.GameProviderName LIKE '%Kiron%'
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Greyhound Racing%'   --DigitainVirtualSport, Pragmatic
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Penalty Shootout%'   -- Pragmatic
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Flat Horse Racing%'   -- Pragmatic
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Fantastic League%'     -- Pragmatic
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Force 1 Racing%'     -- Pragmatic
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Steeplechase%'     -- Pragmatic
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Virtual Football Pro%'     -- 1x2
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Spino Greyhounds%'     -- 1x2
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Spino Horses%'     -- 1x2
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Virtual Challenge Cup%'     -- 1x2
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Virtual Cup%'     -- 1x2
					THEN 'Virtual Games'
				WHEN #a.Name_en LIKE '%Virtual Euro Trophy%'     -- 1x2
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%DigitainVirtualSport%'
					and #a.Name_en LIKE '%Virtual Keno%'
					THEN 'TVGames'
				WHEN #a.GameProviderName LIKE '%DigitainVirtualSport%'
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%GlobalBet%'
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%RFIDgames%'
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%GoldenRace%'
					and #a.Name_en in ('Keno','Keno Deluxe')
					THEN 'TVGames'
				WHEN #a.GameProviderName LIKE '%Golden%'
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%EventBet%'
					THEN 'Poker'
				WHEN #a.GameProviderName LIKE '%Betgames%'
					THEN 'TVGames'
				WHEN #a.GameProviderName LIKE '%Atmosfera%'
				AND #a.Name_en in ('Bingo 37','Keno','Bingo 38')
					THEN 'TVGames'
				WHEN #a.GameProviderName = 'FLG'
				AND #a.Name_en in ('Keno Gold')
					THEN 'TVGames'
				WHEN #a.GameProviderName LIKE '%Betongames%'
					THEN 'Betongames'
				WHEN #a.GameProviderName LIKE '%Evolution%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%LiveCasino%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%EzugiOriginal%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%LuckyStreak%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%VivoGaming%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%NetEntLiveCasino%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%TV Bet%'
					THEN 'TVGames'
				WHEN #a.GameProviderName LIKE '%BetGames%'
					THEN 'TVGames'
				WHEN #a.GameCategoryName LIKE '%P2P%'
					THEN 'P2P'
				WHEN #a.GameProviderName LIKE '%GGPoker%'
					THEN 'Poker'
				WHEN #a.GameCategoryName LIKE '%Slot%'
					THEN 'Slots'
				WHEN #a.GameProviderName LIKE '%InBet%'
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%Leap%'
					THEN 'Virtual Games'
				WHEN #a.GameProviderName LIKE '%Digitain Live Casino%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Digitain%'
					THEN 'Sport'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.GameCategoryName LIKE '%Baccarat%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.GameCategoryName LIKE '%Blackjack%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.GameCategoryName LIKE '%Roulette%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.GameCategoryName LIKE '%TV Show%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.GameCategoryName LIKE '%GameShow%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.Name_en LIKE '%Mega Sic Bo%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.Name_en LIKE '%Dragon Tiger%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Pragmatic%'
					AND #a.Name_en LIKE '%Andar Bahar%'
					THEN 'Live Casino'
				WHEN #a.Name_en LIKE '%Dynamic Roulette 120x%'  --EGT
					THEN 'Live Casino'
				WHEN #a.Name_en LIKE '%Live European Roulette%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Atmosfera%'
					AND #a.Name_en LIKE '%Live Roulette%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Atmosfera%'
					AND #a.Name_en LIKE '%Music Wheel%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%Atmosfera%'
				 AND #a.Name_en LIKE '%Auto Roulette%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%XPG%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%EGT%'
					AND #a.Name_en LIKE '%Onyx Roulette%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%EGT%'
					AND #a.Name_en LIKE '%Live Speed Roulette%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%EGT%'
					AND #a.Name_en LIKE '%Vegas Roulette 500x%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%EGT%'
					AND #a.Name_en LIKE '%Live Roulette – Russian%'
					THEN 'Live Casino'
				WHEN #a.GameProviderName LIKE '%PlayngoArmenia%'
					AND #a.Name_en LIKE '%European Roulette Pro%'
					THEN 'Slots'
				WHEN #a.GameProviderName LIKE '%EGT%'
					AND #a.Name_en LIKE '%Burning Keno Plus%'
					THEN 'Slots'
				WHEN #a.GameProviderName LIKE '%EGT%'
					AND #a.Name_en LIKE '%European Roulette%'
					THEN 'Slots'
				WHEN #a.GameProviderName LIKE '%1x2%'
					AND #a.Name_en LIKE '%Instant Football%'
					THEN 'Virtual Games'

				ELSE 'Other'
    END) Type
    FROM #a ) #b ON #cas.GameID = #b.GameID
    GROUP BY #cas.PartnerUserId, #b.Type

    UNION ALL

    SELECT u.PartnerUserID AS TOTOID,
        COUNT(o.OrderID) AS Order_count,
        SUM(o.OrderAmount) AS Order_amount,
        'Sports' AS Category
    FROM VIEW_sport_PartnerUser_TotogamingAm u
    INNER JOIN VIEW_sport_Orders_TotogamingAm o ON o.UserID = u.UserID
    WHERE o.OrderDate_DT >= '2022-12-01'
    AND o.OrderStateID IN (2, 3, 5, 6)
    AND u.UserTypeID NOT IN (1, 20, 3, 21)
    AND u.UserName NOT LIKE 'test%'
    AND u.UserName NOT LIKE '%TestClient%'
    GROUP BY u.PartnerUserId) AS bc
    ORDER BY bc.TOTOID
    """
    df = pd.read_sql(query, cnx, index_col='TOTOID')
    cnx.close()
    return df
def get_game_attributes(df):
    category_arr = []
    for i in df.index.unique():
        temp = df[df.index == i]
        if len(temp) == 1:
            category = temp.iloc[0, 2]
            category_set = {'TOTOID': i, 'Category': category, 'One_game': True, 'Second_game': None}
            category_arr.append(category_set)
        else:
            largest_amount = temp['Bet_amount'].nlargest(1).values[0]
            second_largest = temp['Bet_amount'].nlargest(2).values[1]
            thresh = sum(temp['Bet_amount']) * 0.7
            fav = temp.query(f"`Bet_amount` == {largest_amount}").iloc[0, 2]
            second_fav = temp.query(f"`Bet_amount` == {second_largest}").iloc[0, 2]
            if largest_amount >= thresh:
                category_set = {'TOTOID': i, 'Category': fav, 'One_game': True, 'Second_game': second_fav}
                category_arr.append(category_set)
            else:
                category_set = {'TOTOID': i, 'Category': fav, 'One_game': False, 'Second_game': second_fav}
                category_arr.append(category_set)
    df_ = pd.DataFrame.from_dict(category_arr)
    df_.set_index('TOTOID', inplace=True)
    return df_

In [3]:
games = get_game_category()

In [4]:
games.head()

,Bet_count,Bet_amount,Category
TOTOID,,,
100377550,15,2600.00,Live Casino
100377550,459,64061.00,Sports
100377550,1637,10824.00,Betongames
100377550,467,6251.72,P2P
100377550,20682,680.00,Slots


In [5]:
attributes = get_game_attributes(games)


KeyboardInterrupt



In [ ]:
attributes.head()